# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [1]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
val = 0
generator = square_root_generator(limit)

for sqrt_value in generator:
  print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [2]:
# Question 1: What is the sum of the outputs of the generator for limit = 5?
limit = 5
solution = 0
generator = square_root_generator(limit)

for sqrt_value in generator:
  solution += sqrt_value

print(f"The {solution = :.3f}")

The solution = 8.382


In [3]:
# Question 2: What is the 13th number yielded from the generator?

limit = 13
solution = 0
generator = square_root_generator(limit)

for idx, num in enumerate(generator):
  if (idx + 1) == 13:
    print(f"The 13th number yielded from generator is {num:.3f}.")

The 13th number yielded from generator is 3.606.


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [4]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [5]:
%%capture
## Install dlt with duckdb dependency
!pip install dlt[duckdb]
import dlt
import duckdb

In [6]:
# Create a pipeline that loads data into duckdb with a particular dataset name
generators_pipeline = dlt.pipeline(
    destination='duckdb',
    dataset_name='people_tables'
)

# Use the generator as the information source, and create a table.
# If the table exists, replace it (avoids loading the same data multiple times)
info = generators_pipeline.run(
    people_1(),
    table_name="people",
    write_disposition="replace"
)

display(info)

LoadInfo(pipeline=<dlt.pipeline.pipeline.Pipeline object at 0x7ad9d9852290>, metrics={'1708025576.1615522': [{'started_at': DateTime(2024, 2, 15, 19, 32, 56, 600487, tzinfo=Timezone('UTC')), 'finished_at': DateTime(2024, 2, 15, 19, 32, 57, 119488, tzinfo=Timezone('UTC'))}]}, destination_type='dlt.destinations.duckdb', destination_displayable_credentials='duckdb:////content/dlt_colab_kernel_launcher.duckdb', destination_name='duckdb', environment=None, staging_type=None, staging_name=None, staging_displayable_credentials=None, destination_fingerprint='', dataset_name='people_tables', loads_ids=['1708025576.1615522'], load_packages=[LoadPackageInfo(load_id='1708025576.1615522', package_path='/var/dlt/pipelines/dlt_colab_kernel_launcher/load/loaded/1708025576.1615522', state='loaded', schema=Schema dlt_colab_kernel_launcher at 135076037200048, schema_update={'people': {'name': 'people', 'columns': {'id': {'name': 'id', 'data_type': 'bigint', 'nullable': True}, 'name': {'name': 'name', 'da

In [7]:
# Create a connection to duckdb using the pipeline name (a property of dlt pipeline)
conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# Grab all the tables that now exist in duckdb to ensure the table loaded as intended
# Use pipeline dataset_name, set in the previous cell
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# Take a look at the table to verify that there are no duplicates
people_tbl = conn.sql("SELECT * FROM people")
display(people_tbl)

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
└─────────────────────┘

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708025576.1615522 │ hOq3UeTWzR1K/w │
│     2 │ Person_2 │    27 │ City_A  │ 1708025576.1615522 │ /pjg2DmjiHxDXA │
│     3 │ Person_3 │    28 │ City_A  │ 1708025576.1615522 │ xmo7HgP+r+fSmA │
│     4 │ Person_4 │    29 │ City_A  │ 1708025576.1615522 │ mi4DAGkzGAoDrA │
│     5 │ Person_5 │    30 │ City_A  │ 1708025576.1615522 │ lAc0hJ1rS6VnAw │
└───────┴──────────┴───────┴─────────┴────────────────────┴────────────────┘

In [8]:
# Append the second generator to the same table as the first
generators_pipeline.run(
    people_2(),
    table_name="people",
    write_disposition="append"
)

# Sanity check to ensure people_2 was appended to people_01 as intended
people_tbl2 = conn.sql("SELECT * FROM people")
print()
display(people_tbl2)

┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708025576.1615522 │ hOq3UeTWzR1K/w │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708025576.1615522 │ /pjg2DmjiHxDXA │ NULL       │
│     3 │ Person_3 │    28 │ City_A  │ 1708025576.1615522 │ xmo7HgP+r+fSmA │ NULL       │
│     4 │ Person_4 │    29 │ City_A  │ 1708025576.1615522 │ mi4DAGkzGAoDrA │ NULL       │
│     5 │ Person_5 │    30 │ City_A  │ 1708025576.1615522 │ lAc0hJ1rS6VnAw │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708025577.2302227 │ Gs2h+LTf+WxgQA │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708025577.2302227 │ 0CtXRYVnCGHSJg │ Job_4      │
│     5 │ 

In [9]:
# Use SQL to grab the sum of all the ages of the people captured in the table
sum_ages = conn.sql("""
                    SELECT SUM(age)
                    FROM people
                    """)
display(sum_ages)

┌──────────┐
│ sum(age) │
│  int128  │
├──────────┤
│      353 │
└──────────┘

# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


# Solution

In [10]:
# Create a new table with merge disposition, making "id" the primary key
generators_pipeline.run(
    people_1(),
    table_name="solution",
    write_disposition="merge",
    primary_key="id"
)

# Display the solution table for later comparison
solution_original = conn.sql("SELECT * FROM solution ORDER BY name;")
display(solution_original)

# Load the second generator with merge disposition
generators_pipeline.run(
    people_2(),
    table_name="solution",
    write_disposition="merge",
    primary_key="id"
)

# Display the solution table to verify changes
solution_merged = conn.sql("SELECT * FROM solution ORDER BY name;")
print("""
----After Merge----
""")
display(solution_merged)

┌───────┬──────────┬───────┬─────────┬───────────────────┬────────────────┐
│  id   │   name   │  age  │  city   │   _dlt_load_id    │    _dlt_id     │
│ int64 │ varchar  │ int64 │ varchar │      varchar      │    varchar     │
├───────┼──────────┼───────┼─────────┼───────────────────┼────────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708025577.984617 │ O8szVFXcxp+luQ │
│     2 │ Person_2 │    27 │ City_A  │ 1708025577.984617 │ 71g4nlQudCXuUw │
│     3 │ Person_3 │    28 │ City_A  │ 1708025577.984617 │ anTddLKZSKEKjg │
│     4 │ Person_4 │    29 │ City_A  │ 1708025577.984617 │ aEXzLrZFQ/wzfg │
│     5 │ Person_5 │    30 │ City_A  │ 1708025577.984617 │ mWMnXOWbMEbl8A │
└───────┴──────────┴───────┴─────────┴───────────────────┴────────────────┘


----After Merge----



┌───────┬──────────┬───────┬─────────┬────────────────────┬────────────────┬────────────┐
│  id   │   name   │  age  │  city   │    _dlt_load_id    │    _dlt_id     │ occupation │
│ int64 │ varchar  │ int64 │ varchar │      varchar       │    varchar     │  varchar   │
├───────┼──────────┼───────┼─────────┼────────────────────┼────────────────┼────────────┤
│     1 │ Person_1 │    26 │ City_A  │ 1708025577.984617  │ O8szVFXcxp+luQ │ NULL       │
│     2 │ Person_2 │    27 │ City_A  │ 1708025577.984617  │ 71g4nlQudCXuUw │ NULL       │
│     3 │ Person_3 │    33 │ City_B  │ 1708025578.8717961 │ yBYakXTgkE+wkQ │ Job_3      │
│     4 │ Person_4 │    34 │ City_B  │ 1708025578.8717961 │ /s0AXqJzTWo55w │ Job_4      │
│     5 │ Person_5 │    35 │ City_B  │ 1708025578.8717961 │ AKVqz7UE0oou6Q │ Job_5      │
│     6 │ Person_6 │    36 │ City_B  │ 1708025578.8717961 │ /Eivo2rUVe08Fg │ Job_6      │
│     7 │ Person_7 │    37 │ City_B  │ 1708025578.8717961 │ 9IDpBO3wclTtJA │ Job_7      │
│     8 │ 

Questions? difficulties? We are here to help.
- DTC data engineering course channel: https://datatalks-club.slack.com/archives/C01FABYF2RG
- dlt's DTC cohort channel: https://dlthub-community.slack.com/archives/C06GAEX2VNX